In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#References
#https://stackoverflow.com/questions/46910206/extracting-bounding-boxes-and-category-labels-in-ms-coco-dataset
#https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/coco.py

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

import tensorflow as tf
import collections
import random
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
import pandas as pd
from google.colab import drive
import numpy as np
# Mounting the drive
drive.mount('/content/drive')

In [ ]:
#littleCo = COCO('/home/r.bohare/coco_data/annotations/instances_train2014.json')

In [ ]:
# Download caption annotation files
annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder):
  annotation_zip = tf.keras.utils.get_file('captions.zip',
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                          extract = True)
  annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'
  instances_file = os.path.dirname(annotation_zip)+'/annotations/instances_train2014.json'
  os.remove(annotation_zip)

In [ ]:
########################Use Pycocotools################################
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

littleCo = COCO(instances_file)



In [ ]:
__BACKPACK_CATEGORY__ = 27 #eg; 27 is backpack
bag_img_ids = littleCo.getImgIds(catIds=[__BACKPACK_CATEGORY__])  #3924
bag_imgs = littleCo.loadImgs(ids=bag_img_ids)
bag_img_names = [image['file_name'] for image in bag_imgs]

prefix_str = '/content/train2014/'
# Append suffix / prefix to strings in list
final_bag_img_names = [prefix_str + sub for sub in bag_img_names]
print("FINAL Bag Images:", len(final_bag_img_names), final_bag_img_names[:2])

train_image_paths = final_bag_img_names
print(len(train_image_paths))

########################Use Pycocotools################################

In [ ]:
with open(annotation_file, 'r') as f:
  annotations = json.load(f)

# Group all captions together having the same image ID.  DEFAULT DICT CREATES KEY IF IT DOES NOT EXIST!
# So this will have IMAGE PATH as KEY and an ARRAY OF CAPTIONS.
image_path_to_caption = collections.defaultdict(list)

for val in annotations['annotations']:
  caption = f"{val['caption']}"
  image_path = '/content/train2014/' + 'COCO_train2014_' + '%012d.jpg' % (val['image_id'])
  image_path_to_caption[image_path].append(caption)


full_image_ids = []
full_image_paths = []
full_captions = []
for index, image_path in enumerate(train_image_paths):
  caption_list = image_path_to_caption[image_path]
  full_captions.extend(caption_list)   #extend IS adding one list to another list.
  full_image_paths.extend([image_path] * len(caption_list))  #ADD IMAGE PATH THAT MANY TIMES DEPENDING UPON HOW MANY CAPTIONS WAS AVAILABLE FOR THAT IMAGE
  full_image_ids.extend([bag_img_ids[index]] * len(caption_list))



In [ ]:
df = pd.DataFrame(full_image_ids, columns=['Image_Id'])
df["Images_Path"] = full_image_paths
df["Caption"] = full_captions
df.to_csv('/content/drive/My Drive/Colab Notebooks/_IMAGE CAPTIONING/BACKPACK_IMAGES_CAPTIONS.csv', index=False)